In [1]:
import numpy as np
import pandas as pd
import random
import torch

Lets dynamically choose type of device to use for our computations. To run on your own GPU one needs to install pytorch and cuda-toolkit.

In [2]:
print(torch.cuda.is_available())
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

True


In [3]:
torch.cuda.get_device_properties(device)

/home/antoni_valls/CV/Kvasir-Semantic-Segmentation/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


_CudaDeviceProperties(name='NVIDIA GeForce RTX 2080 Ti', major=7, minor=5, total_memory=10824MB, multi_processor_count=68)

Define the seed for reproductivity:

In [4]:
def set_seed(seed, use_gpu = True):
    """
    Set SEED for PyTorch reproducibility
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if use_gpu:
        torch.cuda.manual_seed_all(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

SEED = 196
USE_SEED = True
if USE_SEED:
    set_seed(SEED, torch.cuda.is_available())

**Load the images and masks**

In [5]:
augmented = input("Data augmentation? (y/n)")

if augmented == "y":
    train_images = np.load("data/data/train_augmented/combined_images.npy")
    train_masks = np.load("data/data/train_augmented/combined_masks.npy")

elif augmented == "n":
    train_images = np.load("data/data/train_nonaugmented/train_images.npy")
    train_masks = np.load("data/data/train_nonaugmented/train_masks.npy")

else:
    raise ValueError(f"Only 'y' or 'n' ")

test_images = np.load("data/data/test/test_images.npy")
test_masks = np.load("data/data/test/test_masks.npy")

Data augmentation? (y/n) y


In [6]:
print("Train images shape:", train_images.shape)
print("Test images shape:", test_images.shape)
print("Train masks shape:", train_masks.shape)
print("Test masks shape:", test_masks.shape)

Train images shape: (3500, 256, 256, 3)
Test images shape: (300, 256, 256, 3)
Train masks shape: (3500, 256, 256, 1)
Test masks shape: (300, 256, 256, 1)


**Standaritzation**

In [7]:
mean = np.mean(train_images, axis = (0,1,2)) / 255
std = np.std(train_images, axis = (0,1,2)) / 255

print("-----  NORMALIZATION VALUES  -----")
print(f"Mean (RGB): {mean}")
print(f"Standard Deviation (RGB): {std}")

-----  NORMALIZATION VALUES  -----
Mean (RGB): [0.56289435 0.33033543 0.24323519]
Standard Deviation (RGB): [0.31274417 0.23138525 0.19586279]


In [8]:
import torchvision
import torchvision.transforms as transforms

transforms = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),  # Convert the image to a PyTorch tensor and normalize it between [0, 1]
    torchvision.transforms.Normalize(mean, std)  # Normalize the tensor using the provided mean and standard deviation
])

**Validation set**

In [9]:
from sklearn.model_selection import train_test_split

#Validation set (40%) of the testing images
testing_images, val_images, testing_masks, val_masks = train_test_split(
    test_images, test_masks, test_size=0.4, random_state=42)

# Check shape of training images and masks
print("Training Images Shape:", train_images.shape)
print("Training Masks Shape:", train_masks.shape)

# Check shape of validation images and masks
print("Validation Images Shape:", val_images.shape)
print("Validation Masks Shape:", val_masks.shape)

# Check length of training and validation sets
print("Number of Train Examples:", len(train_images))
print("Number of Validation Examples:", len(val_images))
print("Number of Test Examples:", len(testing_images))

Training Images Shape: (3500, 256, 256, 3)
Training Masks Shape: (3500, 256, 256, 1)
Validation Images Shape: (120, 256, 256, 3)
Validation Masks Shape: (120, 256, 256, 1)
Number of Train Examples: 3500
Number of Validation Examples: 120
Number of Test Examples: 180


In [10]:
from src.preprocess import KvasirDataset

# Create KvasirDataset objects for train, validation and test sets
train_dataset = KvasirDataset(images=train_images, masks=train_masks, transforms=transforms)
val_dataset = KvasirDataset(images=val_images,masks=val_masks, transforms=transforms)
test_dataset = KvasirDataset(images=testing_images, masks=testing_masks, transforms=transforms) 


**Define the model**

In [11]:
config = {}
config['name']='DiceLoss_try_Aug'
config['epochs']=100
config['batch_size']=16
config['arch']='NestedUNet'
config['deep_supervision']=True
config['input_channels']=3
config['num_classes']=1
config['early_stopping']=10 # 10 epochs without improving the dice coefficient
#config['input_w']=128
#config['input_h']=128


In [12]:
from torch.utils.data import DataLoader

train_iterator = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_iterator = DataLoader(test_dataset, batch_size=config['batch_size'])
val_iterator = DataLoader(val_dataset,batch_size=config['batch_size'])

In [13]:
from src.main import NestedUNet
from src.utils import count_parameters

model = NestedUNet(config)
print(f"The model has {count_parameters(model):,} trainable parameters.")

The model has 2,264,899 trainable parameters.


In [14]:
model = model.to(device)

In [15]:
from src.loss import DiceLoss, IoULoss, BCEDiceLoss
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion = DiceLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4) 
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=5, verbose=True)

/home/antoni_valls/CV/Kvasir-Semantic-Segmentation/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
from src.train_val_test import train, evaluate
from collections import OrderedDict

log = OrderedDict([
    ('epoch', []),
    ('loss', []),
    ('iou', []),
    ('dice',[]),
    ('accuracy',[]),
    ('val_loss', []),
    ('val_iou', []),
    ('val_dice',[]),
    ('val_accuracy',[])
])

best_dice = 0
trigger = 0
for epoch in range(config['epochs']):
    print('Epoch [%d/%d]' % (epoch, config['epochs']))

    # train for one epoch
    train_log = train(config, model, train_iterator, criterion, optimizer, config['deep_supervision'], device)
    # evaluate on validation set
    val_log = evaluate(config, model, val_iterator, criterion, config['deep_supervision'], device)

    print('loss %.4f - iou %.4f - dice %.4f - accuracy %.4f - val_loss %.4f - val_iou %.4f - val_dice %.4f - val_accuracy %.4f'
          % (train_log['loss'], train_log['iou'], train_log['dice_coef'], train_log['accuracy'],
             val_log['loss'], val_log['iou'], val_log['dice_coef'], val_log['accuracy']))

    log['epoch'].append(epoch)
    log['loss'].append(train_log['loss'])
    log['iou'].append(train_log['iou'])
    log['dice'].append(train_log['dice_coef'])
    log['accuracy'].append(train_log['accuracy'])
    log['val_loss'].append(val_log['loss'])
    log['val_iou'].append(val_log['iou'])
    log['val_dice'].append(val_log['dice_coef'])
    log['val_accuracy'].append(val_log['accuracy'])
    
    pd.DataFrame(log).to_csv('models/%s/log.csv' %
                             config['name'], index=False)

    trigger += 1

    if val_log['dice_coef'] > best_dice:
        torch.save(model.state_dict(), 'models/%s/model.pth' %
                   config['name'])
        best_dice = val_log['dice_coef']
        print("=> saved best model")
        trigger = 0

    # Reduce learning rate when validation metric stops improving
    scheduler.step(val_log['loss'])

    # early stopping
    if config['early_stopping'] >= 0 and trigger >= config['early_stopping']:
        print("=> early stopping")
        break

    torch.cuda.empty_cache()

Epoch [0/100]


100%|█████████████████████████████| 8/8 [00:00<00:00,  9.89it/s, loss=0.663, iou=0.202, dice_coef=0.335, accuracy=0.688]


loss 0.6775 - iou 0.2035 - dice 0.3364 - accuracy 0.5521 - val_loss 0.6627 - val_iou 0.2016 - val_dice 0.3349 - val_accuracy 0.6875
=> saved best model
Epoch [1/100]


100%|█████████████████████████████| 8/8 [00:00<00:00,  9.49it/s, loss=0.634, iou=0.234, dice_coef=0.379, accuracy=0.727]


loss 0.6326 - iou 0.2403 - dice 0.3857 - accuracy 0.6275 - val_loss 0.6340 - val_iou 0.2340 - val_dice 0.3787 - val_accuracy 0.7267
=> saved best model
Epoch [2/100]


...

100%|█████████████████████████████| 8/8 [00:00<00:00,  8.08it/s, loss=0.303, iou=0.695, dice_coef=0.819, accuracy=0.933]


loss 0.2584 - iou 0.7946 - dice 0.8845 - accuracy 0.9503 - val_loss 0.3027 - val_iou 0.6949 - val_dice 0.8186 - val_accuracy 0.9329
=> saved best model
Epoch [26/100]


100%|█████████████████████████████| 8/8 [00:00<00:00,  8.13it/s, loss=0.299, iou=0.699, dice_coef=0.821, accuracy=0.935]


loss 0.2531 - iou 0.8051 - dice 0.8911 - accuracy 0.9525 - val_loss 0.2986 - val_iou 0.6988 - val_dice 0.8206 - val_accuracy 0.9354
=> saved best model
Epoch [27/100]


 63%|███████████████▋         | 137/219 [00:57<00:33,  2.43it/s, loss=0.248, iou=0.816, dice_coef=0.898, accuracy=0.954]

In [ ]:
test_log = evaluate(config, model, test_iterator, criterion, config['deep_supervision'], device)

pd.DataFrame([test_log]).to_csv('models/%s/test_log.csv' % config['name'])

print('test_loss %.4f - test_iou %.4f - test_dice %.4f - test_accuracy %.4f'
          % (test_log['loss'], test_log['iou'], test_log['dice_coef'], test_log['accuracy']))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('models/DiceLoss_try_Aug/log.csv')
test = pd.read_csv('models/DiceLoss_try_Aug/test_log.csv')

# Test values for loss, IoU, and Dice coefficients
test_loss = test['loss'].item()  
test_iou = test['iou'].item() 
test_dice = test['dice_coef'].item() 
test_accuracy = test['accuracy'].item() 

# Create a figure and subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 12))

# Plot the loss features in the first subplot
ax1.plot(df['epoch'], df['loss'], label='Training Loss')
ax1.plot(df['epoch'], df['val_loss'], label='Validation Loss')
ax1.axhline(y=test_loss, color='r', linestyle='--', label='Test Loss')
ax1.annotate(f'{test_loss:.2f}', xy=(90, test_loss+0.005), color='r')
ax1.set_title('Loss Metrics')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()

ax2.plot(df['epoch'], df['accuracy'], label='Training Accuracy')
ax2.plot(df['epoch'], df['val_accuracy'], label='Validation Accuracy')
ax2.axhline(y=test_accuracy, color='r', linestyle='--', label='Test Accuracy')
ax2.annotate(f'{test_accuracy:.2f}', xy=(90, test_accuracy+0.002), color='r')
ax2.set_title('Accuracy Metrics')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

ax3.plot(df['epoch'], df['iou'], label='Training IoU')
ax3.plot(df['epoch'], df['val_iou'], label='Validation IoU')
ax3.axhline(y=test_iou, color='r', linestyle='--', label='Test IoU')
ax3.annotate(f'{test_iou:.2f}', xy=(90, test_iou+0.003), color='r')
ax3.set_title('IoU Metrics')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('IoU')
ax3.legend()

ax4.plot(df['epoch'], df['dice'], label='Training Dice')
ax4.plot(df['epoch'], df['val_dice'], label='Validation Dice')
ax4.axhline(y=test_dice, color='r', linestyle='--', label='Test Dice')
ax4.annotate(f'{test_dice:.2f}', xy=(90, test_dice+0.005), color='r')
ax4.set_title('Dice Metrics')
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Dice')
ax4.legend()

fig.suptitle("UNET++ with DiceLoss and with Data Augmentation")
# Adjust the spacing between subplots
plt.tight_layout()

plt.savefig("results/DiceLoss_Aug.png")
# Display the plot
plt.show()